# load package

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import warnings
warnings.simplefiler("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
from haversine import haversine
import googlemaps
import scipy.stats as stats

from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.python.client import device_lib

from math import sqrt



# load meta data and merge

In [ ]:
air_meta = pd.read_csv('/content/drive/MyDrive/미세먼지/AIR/air_location.csv')



In [ ]:
def merge_data(year):
    meta_all = pd.read_csv('META/meta_all.csv')
    asos = pd.read_csv('ASOS_2012-2020/ASOS_%d.csv' % (year), encoding='cp949')
    aws = pd.read_csv('AWS_2012-2020/AWS_%d.csv' % (year), encoding='utf-8')
    globals()['{}_{}'.format('concat', year)] = pd.concat([asos,aws], axis=0)
    final_concat = pd.merge(globals()['concat_{}'.format(year)], meta_all, on='지점', how='left')
    final_concat.to_csv('weather_2012-2020/weather_{}.csv'.format(year), index=False)
    print('{} shape'.format(year), final_concat.shape)
    print(final_concat.위도.isnull().sum())

for year in range(2012,2021):
    merge_data(year)

## format

In [ ]:
meta_all['위도'] = meta_all['위도'].astype('float')
meta_all['경도'] = meta_all['경도'].astype('float')

In [ ]:
meta_asos = pd.read_csv('META_관측지점정보_ASOS.csv',encoding = 'cp949')
meta_aws = pd.read_csv('META_관측지점정보_AWS.csv',encoding = 'cp949')

wrong_loc = [565,567,599,646,670,932]

meta_aws.drop(columns = ['기압계(관측장비지상높이(m))', '기온계(관측장비지상높이(m))', 
                         '풍속계(관측장비지상높이(m))','강우계(관측장비지상높이(m))'],inplace = True)

aws_arr = meta_aws.values

for i in range(len(aws_arr)):
    if aws_arr[i,0] in wrong_loc:
        aws_arr[i,6] = aws_arr[i,7]
        aws_arr[i,7] = aws_arr[i,8]

meta_aws2 = pd.DataFrame(aws_arr, columns = ['지점', '시작일', '종료일', '지점명', '지점주소', '관리관서', '위도', '경도', '노장해발고도(m)'])
meta_aws2.info()
meta_aws2.drop(columns = ['노장해발고도(m)'], inplace = True )

# ASOS 종료된 관측소 삭제
is_due = meta_asos[meta_asos['종료일'].isnull() == False]
is_due['year'] = is_due['종료일'].apply(lambda x : int(x[:4]))
is_due = is_due[is_due['year'] < 2014]
due_idx = is_due.index.to_list()
meta_asos.drop(due_idx,axis = 0,inplace = True)
meta_asos.reset_index(drop=True, inplace=True)

is_due = meta_aws[meta_aws['종료일'].isnull() == False]
is_due['year'] = is_due['종료일'].apply(lambda x : int(x[:4]))
is_due = is_due[is_due['year'] < 2014]
due_idx = is_due.index.to_list()
meta_aws2.drop(due_idx,axis = 0,inplace = True)
meta_aws2.reset_index(drop=True, inplace=True)

meta_aws2.drop(['종료일'], axis=1, inplace=True)
meta_asos.drop(['종료일'], axis=1, inplace=True)
print(meta_aws2.shape, meta_asos.shape)

meta_all = pd.concat([meta_aws2,meta_asos], axis=0, ignore_index=True)
meta_all.drop_duplicates(['지점'],keep = 'first',inplace = True)
meta_all.reset_index(drop=True, inplace=True)

meta_all.drop(columns = ['시작일','관리관서','노장해발고도(m)',
       '기압계(관측장비지상높이(m))', '기온계(관측장비지상높이(m))', '풍속계(관측장비지상높이(m))',
       '강우계(관측장비지상높이(m))'],inplace = True)

meta_all.info()



In [ ]:
def find_loc(df1, df2):
    df1.insert(3,'기상위도',0)
    df1.insert(4,'기상경도',0)
    
    air_arr = df1[['위도','경도']].values
    weather_arr = df2[['위도','경도']].values
    
    for n1 in range(len(air_arr)):
        distance = 50000000000 
        air_obs = air_arr[n1]
    
    for n2 in range(len(weather_arr)):
        weather_obs = weather_arr[n2]
        
        ## 거리계산 단위는 km
        check = float(haversine(air_obs, weather_obs, unit = 'km'))
        
        if check < distance:
            distance = check
            df1.loc[n1,['기상위도','기상경도']] = weather_obs[0], weather_obs[1]
            df1.loc[n1,'거리'] = distance

  
  return df1
        

## Data mapping

In [ ]:
def loc_mapping(year):
    print('------------ {} ------------'.format(year))
    data = pd.read_csv('air_{}.csv'.format(year))
    print('mapping전 :', data.shape)
    loc = air_location(data)
    
    print('------------------------')
    print('{}년 mapping결과확인'.format(year))
    print(loc.isnull().sum())
    
    air_loc = pd.merge(data,loc,on = ['주소'],how = 'left')
    print('------------------------')
    print('mapping후 :', air_loc.shape)
    
    print('------------------------')
    print(air_loc[['위도','경도']].isnull().sum())
    air_loc.to_csv('AIR_2012-2020/air{}_loc.csv'.format(year), index = False)

In [ ]:
all_loc = pd.DataFrame(columns = ['주소','위도','경도'])
for year in range(2012, 2021):
    data = pd.read_csv('air{}_loc.csv'.format(year))
    loc = data[['주소','위도','경도']]
    
    all_loc = pd.concat([all_loc,loc],axis = 0)
    
    all_loc.drop_duplicates(['주소','위도','경도'],keep='first',inplace = True)



In [ ]:
null_lat = {75:37.3274259,624:37.5817708,
            850: 37.5817708, 869:36.4741724,
            963:36.4741724, 998 : 37.1754027,
            1102:37.5584719}

null_lng = {75:126.9237322, 624:126.863756,
            850:126.863756,869:127.25034,
            963:127.25034, 998 :  127.0407797,
            1102 : 126.5972784}

In [ ]:
for i in null_idx:
    all_loc.loc[i,'위도'] = null_lat[i]
    all_loc.loc[i,'경도'] = null_lng[i]

In [ ]:
all_loc.to_csv('air_location.csv',index = False)

In [ ]:
air_location = pd.read_csv('air_location.csv')
air_location.drop_duplicates(['위도','경도'],keep = 'first',inplace = True)
air_location.reset_index(drop=True, inplace=True)
len(air_location)
all_loc = pd.read_csv('META/merge_dist.csv')
for year in range(2012,2021):
    print('------------ {} ------------'.format(year))
    data = pd.read_csv('AIR_2012-2020/air_{}.csv'.format(year))
    print('mapping전 :', data.shape)
    air_loc = pd.merge(data,all_loc,on = ['주소'],how = 'left')
    print(air_loc[['위도','경도']].isnull().sum())
    print('mapping후 :', data.shape)
    air_loc.to_csv('AIR_2012-2020/air_{}_loc.csv'.format(year), index = False)

## Air data format

In [ ]:
def date_format(year):
    print('------------ {} ------------'.format(year))
    df = pd.read_csv('air_{}_loc.csv'.format(year))
    shape1 = df.shape
    period = len(df.측정일시.unique())
    print('start date check({}010101)'.format(year),df.측정일시.unique()[0])
    datelist = pd.date_range(datetime(int(year), 1, 1, 1), periods=period, freq='H') # 시작시간, period 확인
    a = list(df['측정일시'].unique())
    b = list(datelist.strftime("%Y-%m-%d %H:%M"))
    print('finally modified date({}-01-01 00:00)'.format(year+1),b[-1])
    print('check date:', len(a) == len(b))
    date_dic = { original:datetime for original, datetime in zip(a, b) }
    df['일시'] = df['측정일시'].apply(lambda x: date_dic[x])    
    df.drop(['측정일시'], inplace=True, axis=1)    
    shape2 = df.shape
    print('check shape:', shape1 == shape2)
    df.to_csv('air_{}_loc.csv'.format(year), index=False)
    #return df

In [ ]:
for year in range(2012,2021):
    date_format(year)

## Air + Weather

In [ ]:
def air_weather_merge(year):
    air = pd.read_csv('AIR_2012-2020/air_{}_loc.csv'.format(year))
    wea = pd.read_csv('weather_2012-2020/weather_{}.csv'.format(year))
    all_ = pd.merge(air,wea, left_on = ['일시', '기상위도', '기상경도'], right_on=['일시', '위도', '경도'], how = 'left')
    print('all {} shape:', all_.shape)
    all_.to_csv('ALL_2012-2020/all_{}.csv'.format(year), index=False)

In [ ]:
for year in range(2012,2021):
    air_weather_merge(year)

### drop -999

In [ ]:
def drop_999(year): # SO2	CO	O3	NO2	PM10
    print('-----------{}-----------'.format(year))
    data = pd.read_csv('ALL_2012-2020/all_{}.csv'.format(year))
    shape1 = data.shape
    data_air = data[['SO2','CO','O3','NO2','PM10']]
    drop_index1 = list(data_air[data_air['SO2']< 0].index)
    drop_index2 = list(data_air[data_air['CO']< 0].index)
    drop_index3 = list(data_air[data_air['O3']< 0].index)
    drop_index4 = list(data_air[data_air['NO2']< 0].index)
    drop_index5 = list(data_air[data_air['PM10']< 0].index)

    drop_index = drop_index1 + drop_index2 + drop_index3 + drop_index4 + drop_index5
    drop_index = set(drop_index)
    drop_index = list(drop_index)    
    
    print('삭제된 행 수:',len(drop_index))
    data.drop(drop_index, inplace=True)
    shape2 = data.shape
    print('shape 전후 비교:',shape1, shape2)
    data.to_csv('ALL_2012-2020/all_{}_drop.csv'.format(year), index=False)
    #return data

In [ ]:
for year in range(2012,2021):
    drop_999(year)

In [ ]:
def all_999_drop(year):
    air = pd.read_csv('AIR_2012-2020/air_{}_loc.csv'.format(year))
    wea = pd.read_csv('weather_2012-2020/weather_{}.csv'.format(year))
    all_ = pd.merge(air,wea, left_on = ['일시', '기상위도', '기상경도'], right_on=['일시', '위도', '경도'], how = 'left')
    print('all {} shape:', all_.shape)
    all_.to_csv('ALL_2012-2020/all_{}.csv'.format(year), index=False)

# Preprocess

In [ ]:
all2014 = pd.read_csv('all_2014.csv')
all2015 = pd.read_csv('all_2015.csv')
all2016 = pd.read_csv('all_2016.csv')
all2017 = pd.read_csv('all_2017.csv')
all2018 = pd.read_csv('all_2018.csv')
all2019 = pd.read_csv('all_2019.csv')
all2020 = pd.read_csv('all_2020.csv')

all_data = pd.concat([all2014,all2015,all2016,all2017,all2018,all2019,all2020], axis=0)
all_data.to_csv('all_2014-2020.csv', index=False)

In [ ]:
def merge_data(year):
    #meta_all = pd.read_csv('META/meta_all.csv')
    asos = pd.read_csv('ALL_2012-2020/all_%d.csv' % (year))
    aws = pd.read_csv('ALL_2012-2020/all_%d.csv' % (year))
    globals()['{}_{}'.format('concat', year)] = pd.concat([asos,aws], axis=0)
    final_concat = pd.merge(globals()['concat_{}'.format(year)], meta_all, on='지점', how='left')
    final_concat.to_csv('weather_2012-2020/weather_{}.csv'.format(year), index=False)
    print('{} shape'.format(year), final_concat.shape)
    print(final_concat.위도.isnull().sum())

## Null Imputation

In [ ]:
def find_null(df):
    df['지점'] = df['지점'].astype('object')
    null_feature = []
    for col in df.columns.tolist():
        null_count = df[col].isnull().sum()
        null_ratio = round(null_count/len(df), 2)*100
        if null_ratio >= 50:
            null_feature.append(col)
        print(col, ' : ', round(null_count/len(df), 2)*100 ,'%' )
  
    print('----- null_ratio over 50% : -----')
    print(null_feature)
  
    # null 비율이 50%이상인인 변수들은 제거
    data = df.drop(columns = null_feature)
    return data, null_feature

In [ ]:
data, null_feature = find_null(all_data)

## Outlier

In [ ]:
def plot_out(df):
    
    feature = [i for i in df.columns.to_list() if df[i].dtype == 'float64']
    plt.figure(figsize = (25,15))
    for i in range(len(feature)):
        h = (len(feature) // 4) + 1
        plt.subplot(h,4,i+1)
        sns.boxplot(x = feature[i],data = df)
    plt.show()


def find_out(df):

    out_count = {}
    out_idx = []

    #온도
    temp = df[df['기온(°C)'] < - 100].index.tolist()
    out_idx.extend(temp)
    out_count['기온(°C)'] = len(out_idx)
    df.loc[temp,'기온(°C)'] = np.NaN

    #습도
    humid = df[df['습도(%)'] <= -999.9].index.tolist()
    out_idx.extend(humid)
    out_count['습도(%)'] = len(humid)
    df.loc[humid,'습도(%)'] = np.NaN

    out_idx = list(set(out_idx))
    
    return out_count, out_idx




In [ ]:
plot_out(data2)

## Correlation test

In [ ]:
data2.rename(columns = {'위도_x':'위도_미세먼지','경도_x':'경도_미세먼지'}, inplace = True)
data2.columns
data2.rename(columns = {'위도_미세먼지' : 'lat_air', '경도_미세먼지' : 'lon_air','기상관측위도' : 'lat_weather',
                      '기상관측경도' : 'lon_weather','기온(°C)':'temperature','강수량(mm)' : 'precipitation',
                       '풍속(m/s)' : 'wind_speed','습도(%)':'humidity','풍향(deg)':'wind_direction'},inplace = True)

col = ['PM10', 'SO2', 'CO', 'O3', 'NO2', 'lat_air', 'lon_air',
       'lat_weather', 'lon_weather', 'temperature', 'precipitation',
       'wind_speed', 'wind_direction', 'humidity']
cor_dict = {'Variable' : ['SO2', 'CO', 'O3', 'NO2', 'lat_air', 'lon_air', 'lat_weather',
       'lon_weather', 'temperature', 'precipitation', 'wind_speed',
       'wind_direction', 'humidity'],
           'Correlation value' : [0.27423588695628665, 0.35496693047554745, -0.02369964881321885,
       0.33152344997678673, 0.09084290666082527, -0.056391897151533456,
       0.0906610613277951, -0.05548579820320683, -0.10822153530423906,
       -0.07090482791143295, -0.03533185904073229, 0.03171530421690104,
       -0.1597643906655796]}

cor_df = pd.DataFrame(cor_dict)
cor_df.sort_values(by = 'Correlation value',inplace = True,ascending = False)
plt.figure(figsize = (13,8))
sns.barplot(data=cor_df,y = 'Variable',x = 'Correlation value')
#plt.xticks(rotation = 90)
plt.ylabel('Variable', fontsize = 13)
plt.xlabel('Correlation Coefficient', fontsize = 13)
plt.savefig('correlation graph.png',dpi =200)


plt.show()


plt.figure(figsize=(10,10))
sns.heatmap(data = data[col].corr(), annot=True, fmt = '.2f', linewidths=.5, cmap='Blues')

# Modeling

In [ ]:
train_final = np.load('data/train_final.npy')
x_train, x_valid, y_train, y_valid = train_test_split(train_final[:,1:], train_final[:,0], test_size=0.2,random_state = 42)
x_train.shape, x_valid.shape

x_train = x_train.reshape(x_train.shape[0],1,x_train.shape[1])
x_valid = x_valid.reshape(x_valid.shape[0],1,x_valid.shape[1])

print(x_train.shape, x_valid.shape)

## LSTM

In [ ]:
model = Sequential()
model.add(LSTM(16, 
               input_shape=(x_train.shape[1], x_train.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )

model.add(Dense(1))

In [ ]:
import os

device_name = "/device:GPU:" + '0'

with tf.device(device_name):
    start_time = time.time()
    print('Start')
    model.compile(loss='mean_squared_error', optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', patience=5)

    model_path = 'LSTM_trial1'
    filename = os.path.join(model_path, 'tmp_checkpoint.h5')
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

    history = model.fit(x_train, y_train, 
                    epochs=20, 
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])
    
    print('End')
    print("Time: {:.4f}sec".format((time.time() - start_time)))

## Predict

In [ ]:
model.load_weights(filename)
pred = model.predict(test_feature)

pred.shape